## Ouput: Calculation of Estimated Salary

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import pickle

In [22]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Preprocess the Data

In [23]:
df=df.drop(columns=["RowNumber","CustomerId","Surname"],axis=1)

In [24]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Label Encoder of Gender

In [25]:
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

In [26]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


## One Hot Encoding for Geography

In [27]:
onehot_encoder_geo=OneHotEncoder(sparse_output=False)
geo_encoded=onehot_encoder_geo.fit_transform(df[['Geography']])

In [28]:
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


## Combine With Original DataFrame

In [29]:
df=pd.concat([df.drop(columns=["Geography"],axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


## Split The data into Dependent and Independent

In [30]:
X=df.drop(columns=["EstimatedSalary"],axis=1)
y=df['EstimatedSalary']

## Train Test Split

In [31]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

### Standard Scalar

In [32]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

## Save the encoder and scaler

In [33]:
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("onehot_encoder_geo.pkl","wb")as file:
    pickle.dump(onehot_encoder_geo,file)

with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

## ANN Regression Problem Statement

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Build the model


In [37]:
X_train.shape[1],

(12,)

In [39]:
model=Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)),
    Dense(32,activation="relu"),
    Dense(1) # Default applied acitvation function is Linear Activation Function - > this is for Regression
])

## Compile the Model

In [40]:
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=["mae"])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Logs

In [42]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

## Setup the Tensorboard
log_dir="regressionlogs/fit/"+datetime.datetime.now().strftime("%Y-%m-%d__%H-%M-%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [43]:
## Setup EarlyStopping
early_stopping_callback=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

### Train the Model

In [45]:
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 4ms/step - loss: 100393.2578 - mae: 100393.2578 - val_loss: 98575.0469 - val_mae: 98575.0469
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99834.8203 - mae: 99834.8203 - val_loss: 97437.9375 - val_mae: 97437.9375
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 97836.7266 - mae: 97836.7266 - val_loss: 94496.2734 - val_mae: 94496.2734
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 93822.5078 - mae: 93822.5078 - val_loss: 89434.3047 - val_mae: 89434.3047
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 87788.2812 - mae: 87788.2812 - val_loss: 82615.2500 - val_mae: 82615.2500
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 80233.7188 - mae: 80233.7188 - val_loss: 74786.6641 - val_mae: 74786.6641
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 72137.6797 - mae: 72137.679

In [46]:
%load_ext tensorboard

In [49]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 12548), started 0:00:18 ago. (Use '!kill 12548' to kill it.)

## Evaluate model for the test data

In [50]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print("Test MAE: ",test_mae)
print("Test LOSS: ",test_loss)

63/63 [==============================] - 0s 2ms/step - loss: 50263.8906 - mae: 50263.8906
Test MAE:  50263.890625
Test LOSS:  50263.890625


## Save the model

In [51]:
model.save("regression_model.h5")

c:\PYTHON\Python_Udemy\Projects\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [52]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
